# Libraries

In [0]:

from google.colab import files
uploaded_train = files.upload()
uploaded_test = files.upload()
import zipfile 
import io


Saving firmas_train_thres.zip to firmas_train_thres.zip


Saving firmas_test_thres.zip to firmas_test_thres.zip


In [0]:

data_train = zipfile.ZipFile(io.BytesIO(uploaded_train['firmas_train_thres.zip']),'r')
data_train.extractall()
data_test = zipfile.ZipFile(io.BytesIO(uploaded_test['firmas_test_thres.zip']),'r')
data_test.extractall()


In [0]:
from keras.layers import Input,Dense,Conv2D,MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

In [0]:
img_input = Input(shape=(150,150,1))

In [0]:
#padding puede ser valid o same, valid es que no quieres padding,
#same que si
x = Conv2D(16,(3,3),activation='relu',padding='same')(img_input)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(8,(3,3), activation='relu',padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8,(3,3),activation='relu',padding='same')(x)
encoded = MaxPooling2D((2,2),padding='same')(x)

In [0]:
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

In [0]:
autoencoder = Model(img_input,decoded)
autoencoder.compile(optimizer='adadelta',loss='binary_crossentropy')

In [0]:
train_dataset = ImageDataGenerator()
train = train_dataset.flow_from_directory('/content/firmas_train_thres',
                                          batch_size=32,
                                          target_size=(150,150),
                                          class_mode='input')
test_dataset = ImageDataGenerator()
test = test_dataset.flow_from_directory('/content/firmas_test_thres',
                                          batch_size=32,
                                          target_size=(150,150),
                                          class_mode='input')


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [0]:
# steps per epoch es num_samples / batch_size -> 634/32 = 20
autoencoder.fit_generator(train,epochs=5, steps_per_epoch=20,
                   validation_data=test,validation_steps=8,verbose=1)

Epoch 1/20


KeyboardInterrupt: ignored